# Regression / Classification with sklearn

In [1]:
# Regression with various learner

from Library.Import import *
from Library.Utilities import Linear, MLP, XGB, LeaveXout, read_XY
from sklearn.metrics import r2_score, accuracy_score, f1_score, matthews_corrcoef

DIRECTORY = './'
problempath = f'{DIRECTORY}Dataset_input/Dataset_open_ML/Regression/'
problems = os.listdir(problempath)
xfold = 5
niter = 5

problems = [
    'raw_geographical_origin_of_music', # 0
    'raw_pumadyn32nh',
    'raw_white_wine',
    'raw_kin8nm',
    'raw_cars', # 4
    'raw_airfoil_self_noise', #5
    'raw_QSAR_fish_toxicity', # 6
    'raw_space_ga',
    'raw_concrete_compressive_strength', # 8
    'raw_grid_stability',
    'raw_miami_housing',
    'raw_cpu_activity', # 11
    'raw_naval_propulsion_plant',
    'raw_energy_efficiency' # 13
]
scoring_function = r2_score
for learner in [Linear, MLP, XGB]:
    for j in [11]: # list(range(14)):
        problem = problems[j].split('.')[0]
        filename = f'{problempath}{problem}'
        feature, X, y = read_XY(filename, scaling='XY')
        r2_avr, r2_dev, _ = LeaveXout(X, y.ravel(), feature, 
                                      learner=learner, 
                                      scoring_function=scoring_function, 
                                      xfold=xfold, niter=niter,verbose=False)
        print(f'{problem} Size: {X.shape} Method: {learner.__name__} Score: {scoring_function.__name__} '
              f'R2: {r2_avr:.3f}±{r2_dev:.3f} '
              f'for {xfold}-fold-CV and {niter} iter')


raw_cpu_activity Size: (8192, 21) Method: Linear Score: r2_score R2: 0.722±0.002 for 5-fold-CV and 5 iter
raw_cpu_activity Size: (8192, 21) Method: MLP Score: r2_score R2: 0.980±0.001 for 5-fold-CV and 5 iter
raw_cpu_activity Size: (8192, 21) Method: XGB Score: r2_score R2: 0.983±0.002 for 5-fold-CV and 5 iter


'\nraw_cpu_activity Size: (8192, 21) Method: Linear Score: r2_score R2: 0.722±0.002 for 5-fold-CV and 5 iter\nraw_cpu_activity Size: (8192, 21) Method: MLP Score: r2_score R2: 0.980±0.000 for 5-fold-CV and 5 iter\nraw_cpu_activity Size: (8192, 21) Method: XGB Score: r2_score R2: 0.983±0.002 for 5-fold-CV and 5 iter\n'

In [1]:
# Classification with various learner

from Library.Import import *
from Library.Utilities import bayes_classifier, svm_classifier, XGB, GP, MLP
from Library.Utilities import LeaveXout, read_XY
from sklearn.metrics import r2_score, accuracy_score, f1_score, matthews_corrcoef

DIRECTORY = './'
problempath = f'{DIRECTORY}Dataset_input/Dataset_classification/'
problems = os.listdir(problempath)
xfold = 5
niter = 5

problems = [
    'and', #0
    'or',  
    'linear', #2
    'triangle', #3
    'xor',    
    'checkerboard', #5
    'circle',
    'sine',  #7
    'concentric', #8
    'dots' #9
]

scoring_function = f1_score
for learner in [svm_classifier, bayes_classifier, MLP, XGB]:
    for j in range(10):
        problem = problems[j].split('.')[0]
        filename = f'{problempath}{problem}'
        feature, X, y = read_XY(filename, scaling='XY')
        acc_avr, acc_dev, _ = LeaveXout(X, y.ravel(), feature, 
                                      learner=learner, scoring_function=scoring_function, 
                                      xfold=xfold, niter=niter, verbose=False)
        print(f'{problem} Size: {X.shape} Method: {learner.__name__} Score: {scoring_function.__name__} '
              f'Acc: {acc_avr:.3f}±{acc_dev:.3f} '
              f'for {xfold}-fold-CV and {niter} iter')

and Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.791±0.006 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.947±0.002 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.975±0.003 for 5-fold-CV and 5 iter
triangle Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.870±0.005 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.546±0.039 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.707±0.019 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.000±0.000 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.794±0.006 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.808±0.000 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.000±0.0

With f1-score
and Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.791±0.006 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.947±0.002 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.975±0.003 for 5-fold-CV and 5 iter
triangle Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.870±0.005 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.546±0.039 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.707±0.019 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.000±0.000 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.794±0.006 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.808±0.000 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: svm_classifier Score: f1_score Acc: 0.000±0.000 for 5-fold-CV and 5 iter
and Size: (500, 2) Method: bayes_classifier Score: f1_score Acc: 0.887±0.010 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: bayes_classifier Score: f1_score Acc: 0.969±0.001 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: bayes_classifier Score: f1_score Acc: 0.956±0.004 for 5-fold-CV and 5 iter
triangle Size: (500, 2) Method: bayes_classifier Score: f1_score Acc: 0.876±0.004 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: bayes_classifier Score: f1_score Acc: 0.520±0.021 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: bayes_classifier Score: f1_score Acc: 0.654±0.014 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: bayes_classifier Score: f1_score Acc: 0.582±0.012 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: bayes_classifier Score: f1_score Acc: 0.789±0.006 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: bayes_classifier Score: f1_score Acc: 0.790±0.003 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: bayes_classifier Score: f1_score Acc: 0.000±0.000 for 5-fold-CV and 5 iter
and Size: (500, 2) Method: MLP Score: f1_score Acc: 0.881±0.026 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: MLP Score: f1_score Acc: 0.949±0.001 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: MLP Score: f1_score Acc: 0.995±0.002 for 5-fold-CV and 5 iter
triangle Size: (500, 2) Method: MLP Score: f1_score Acc: 0.869±0.003 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: MLP Score: f1_score Acc: 0.928±0.039 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: MLP Score: f1_score Acc: 0.691±0.009 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: MLP Score: f1_score Acc: 0.734±0.155 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: MLP Score: f1_score Acc: 0.811±0.014 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: MLP Score: f1_score Acc: 0.842±0.011 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: MLP Score: f1_score Acc: 0.308±0.139 for 5-fold-CV and 5 iter
and Size: (500, 2) Method: XGB Score: f1_score Acc: 0.994±0.003 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: XGB Score: f1_score Acc: 0.995±0.002 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: XGB Score: f1_score Acc: 0.962±0.003 for 5-fold-CV and 5 iter
triangle Size: (500, 2) Method: XGB Score: f1_score Acc: 0.950±0.006 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: XGB Score: f1_score Acc: 0.992±0.002 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: XGB Score: f1_score Acc: 0.979±0.005 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: XGB Score: f1_score Acc: 0.929±0.010 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: XGB Score: f1_score Acc: 0.948±0.005 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: XGB Score: f1_score Acc: 0.963±0.002 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: XGB Score: f1_score Acc: 0.912±0.010 for 5-fold-CV and 5 iter

With accuracy_score

and Size: (500, 2) Method: svm_classifier Score: accuracy_score Acc: 0.904±0.004 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: svm_classifier Score: accuracy_score Acc: 0.920±0.004 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: svm_classifier Score: accuracy_score Acc: 0.976±0.005 for 5-fold-CV and 5 iter
triangle Size: (500, 2) Method: svm_classifier Score: accuracy_score Acc: 0.906±0.003 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: svm_classifier Score: accuracy_score Acc: 0.613±0.033 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: svm_classifier Score: accuracy_score Acc: 0.556±0.008 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: svm_classifier Score: accuracy_score Acc: 0.708±0.000 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: svm_classifier Score: accuracy_score Acc: 0.791±0.005 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: svm_classifier Score: accuracy_score Acc: 0.678±0.000 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: svm_classifier Score: accuracy_score Acc: 0.742±0.000 for 5-fold-CV and 5 iter
and Size: (500, 2) Method: bayes_classifier Score: accuracy_score Acc: 0.951±0.004 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: bayes_classifier Score: accuracy_score Acc: 0.953±0.003 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: bayes_classifier Score: accuracy_score Acc: 0.952±0.007 for 5-fold-CV and 5 iter
triangle Size: (500, 2) Method: bayes_classifier Score: accuracy_score Acc: 0.912±0.004 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: bayes_classifier Score: accuracy_score Acc: 0.528±0.018 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: bayes_classifier Score: accuracy_score Acc: 0.556±0.013 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: bayes_classifier Score: accuracy_score Acc: 0.831±0.004 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: bayes_classifier Score: accuracy_score Acc: 0.786±0.005 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: bayes_classifier Score: accuracy_score Acc: 0.673±0.004 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: bayes_classifier Score: accuracy_score Acc: 0.742±0.000 for 5-fold-CV and 5 iter
and Size: (500, 2) Method: MLP Score: accuracy_score Acc: 0.944±0.009 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: MLP Score: accuracy_score Acc: 0.923±0.006 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: MLP Score: accuracy_score Acc: 0.995±0.003 for 5-fold-CV and 5 iter
triangle Size: (500, 2) Method: MLP Score: accuracy_score Acc: 0.909±0.005 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: MLP Score: accuracy_score Acc: 0.928±0.036 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: MLP Score: accuracy_score Acc: 0.565±0.031 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: MLP Score: accuracy_score Acc: 0.866±0.047 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: MLP Score: accuracy_score Acc: 0.805±0.017 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: MLP Score: accuracy_score Acc: 0.758±0.031 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: MLP Score: accuracy_score Acc: 0.763±0.017 for 5-fold-CV and 5 iter
and Size: (500, 2) Method: XGB Score: accuracy_score Acc: 0.997±0.002 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: XGB Score: accuracy_score Acc: 0.993±0.002 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: XGB Score: accuracy_score Acc: 0.960±0.006 for 5-fold-CV and 5 iter
triangle Size: (500, 2) Method: XGB Score: accuracy_score Acc: 0.965±0.005 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: XGB Score: accuracy_score Acc: 0.990±0.004 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: XGB Score: accuracy_score Acc: 0.975±0.006 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: XGB Score: accuracy_score Acc: 0.962±0.006 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: XGB Score: accuracy_score Acc: 0.951±0.005 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: XGB Score: accuracy_score Acc: 0.950±0.005 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: XGB Score: accuracy_score Acc: 0.953±0.005 for 5-fold-CV and 5 iter

With matthews_corrcoef

and Size: (500, 2) Method: svm_classifier Score: matthews_corrcoef Acc: 0.735±0.010 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: svm_classifier Score: matthews_corrcoef Acc: 0.785±0.010 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: svm_classifier Score: matthews_corrcoef Acc: 0.953±0.010 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: svm_classifier Score: matthews_corrcoef Acc: 0.236±0.071 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: svm_classifier Score: matthews_corrcoef Acc: -0.006±0.030 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: svm_classifier Score: matthews_corrcoef Acc: 0.000±0.000 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: svm_classifier Score: matthews_corrcoef Acc: 0.583±0.010 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: svm_classifier Score: matthews_corrcoef Acc: 0.000±0.000 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: svm_classifier Score: matthews_corrcoef Acc: 0.000±0.000 for 5-fold-CV and 5 iter
and Size: (500, 2) Method: bayes_classifier Score: matthews_corrcoef Acc: 0.867±0.010 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: bayes_classifier Score: matthews_corrcoef Acc: 0.876±0.007 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: bayes_classifier Score: matthews_corrcoef Acc: 0.907±0.014 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: bayes_classifier Score: matthews_corrcoef Acc: 0.055±0.037 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: bayes_classifier Score: matthews_corrcoef Acc: 0.066±0.029 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: bayes_classifier Score: matthews_corrcoef Acc: 0.583±0.010 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: bayes_classifier Score: matthews_corrcoef Acc: 0.572±0.011 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: bayes_classifier Score: matthews_corrcoef Acc: 0.125±0.018 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: bayes_classifier Score: matthews_corrcoef Acc: 0.000±0.000 for 5-fold-CV and 5 iter

and Size: (500, 2) Method: svm_classifier Acc: 0.904±0.004 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: svm_classifier Acc: 0.920±0.004 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: svm_classifier Acc: 0.976±0.005 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: svm_classifier Acc: 0.613±0.033 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: svm_classifier Acc: 0.556±0.008 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: svm_classifier Acc: 0.708±0.000 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: svm_classifier Acc: 0.791±0.005 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: svm_classifier Acc: 0.678±0.000 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: svm_classifier Acc: 0.742±0.000 for 5-fold-CV and 5 iter
and Size: (500, 2) Method: bayes_classifier Acc: 0.951±0.004 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: bayes_classifier Acc: 0.953±0.003 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: bayes_classifier Acc: 0.952±0.007 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: bayes_classifier Acc: 0.528±0.018 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: bayes_classifier Acc: 0.556±0.013 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: bayes_classifier Acc: 0.831±0.004 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: bayes_classifier Acc: 0.786±0.005 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: bayes_classifier Acc: 0.673±0.004 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: bayes_classifier Acc: 0.742±0.000 for 5-fold-CV and 5 iter
and Size: (500, 2) Method: MLP Acc: 0.943±0.009 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: MLP Acc: 0.924±0.006 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: MLP Acc: 0.995±0.003 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: MLP Acc: 0.922±0.042 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: MLP Acc: 0.556±0.023 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: MLP Acc: 0.884±0.056 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: MLP Acc: 0.808±0.015 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: MLP Acc: 0.749±0.028 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: MLP Acc: 0.765±0.020 for 5-fold-CV and 5 iter
and Size: (500, 2) Method: XGB Acc: 0.997±0.002 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: XGB Acc: 0.993±0.002 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: XGB Acc: 0.960±0.006 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: XGB Acc: 0.990±0.004 for 5-fold-CV and 5 iter
checkerboard Size: (500, 2) Method: XGB Acc: 0.975±0.006 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: XGB Acc: 0.962±0.006 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: XGB Acc: 0.951±0.005 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: XGB Acc: 0.950±0.005 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: XGB Acc: 0.953±0.005 for 5-fold-CV and 5 iter

Sept. 5 scaler = 32

and Size: (1024, 2) Method: svm_classifier Acc: 0.917±0.002 for 5-fold-CV and 5 iter
or Size: (1024, 2) Method: svm_classifier Acc: 0.917±0.002 for 5-fold-CV and 5 iter
linear Size: (1024, 2) Method: svm_classifier Acc: 0.981±0.003 for 5-fold-CV and 5 iter
xor Size: (1024, 2) Method: svm_classifier Acc: 0.533±0.029 for 5-fold-CV and 5 iter
checkerboard Size: (1024, 2) Method: svm_classifier Acc: 0.554±0.000 for 5-fold-CV and 5 iter
circle Size: (1024, 2) Method: svm_classifier Acc: 0.724±0.000 for 5-fold-CV and 5 iter
sine Size: (1024, 2) Method: svm_classifier Acc: 0.823±0.003 for 5-fold-CV and 5 iter
concentric Size: (1024, 2) Method: svm_classifier Acc: 0.694±0.000 for 5-fold-CV and 5 iter
dots Size: (1024, 2) Method: svm_classifier Acc: 0.762±0.000 for 5-fold-CV and 5 iter
and Size: (1024, 2) Method: bayes_classifier Acc: 0.955±0.002 for 5-fold-CV and 5 iter
or Size: (1024, 2) Method: bayes_classifier Acc: 0.944±0.001 for 5-fold-CV and 5 iter
linear Size: (1024, 2) Method: bayes_classifier Acc: 0.966±0.004 for 5-fold-CV and 5 iter
xor Size: (1024, 2) Method: bayes_classifier Acc: 0.472±0.033 for 5-fold-CV and 5 iter
checkerboard Size: (1024, 2) Method: bayes_classifier Acc: 0.508±0.007 for 5-fold-CV and 5 iter
circle Size: (1024, 2) Method: bayes_classifier Acc: 0.834±0.002 for 5-fold-CV and 5 iter
sine Size: (1024, 2) Method: bayes_classifier Acc: 0.826±0.003 for 5-fold-CV and 5 iter
concentric Size: (1024, 2) Method: bayes_classifier Acc: 0.648±0.002 for 5-fold-CV and 5 iter
dots Size: (1024, 2) Method: bayes_classifier Acc: 0.762±0.000 for 5-fold-CV and 5 iter
and Size: (1024, 2) Method: MLP Acc: 0.973±0.009 for 5-fold-CV and 5 iter
or Size: (1024, 2) Method: MLP Acc: 0.938±0.014 for 5-fold-CV and 5 iter
linear Size: (1024, 2) Method: MLP Acc: 0.999±0.001 for 5-fold-CV and 5 iter
xor Size: (1024, 2) Method: MLP Acc: 0.965±0.025 for 5-fold-CV and 5 iter
checkerboard Size: (1024, 2) Method: MLP Acc: 0.565±0.020 for 5-fold-CV and 5 iter
circle Size: (1024, 2) Method: MLP Acc: 0.933±0.041 for 5-fold-CV and 5 iter
sine Size: (1024, 2) Method: MLP Acc: 0.854±0.017 for 5-fold-CV and 5 iter
concentric Size: (1024, 2) Method: MLP Acc: 0.782±0.028 for 5-fold-CV and 5 iter
dots Size: (1024, 2) Method: MLP Acc: 0.794±0.015 for 5-fold-CV and 5 iter
and Size: (1024, 2) Method: XGB Acc: 1.000±0.000 for 5-fold-CV and 5 iter
or Size: (1024, 2) Method: XGB Acc: 1.000±0.000 for 5-fold-CV and 5 iter
linear Size: (1024, 2) Method: XGB Acc: 0.981±0.004 for 5-fold-CV and 5 iter
xor Size: (1024, 2) Method: XGB Acc: 1.000±0.000 for 5-fold-CV and 5 iter
checkerboard Size: (1024, 2) Method: XGB Acc: 0.994±0.002 for 5-fold-CV and 5 iter
circle Size: (1024, 2) Method: XGB Acc: 0.992±0.002 for 5-fold-CV and 5 iter
sine Size: (1024, 2) Method: XGB Acc: 0.974±0.003 for 5-fold-CV and 5 iter
concentric Size: (1024, 2) Method: XGB Acc: 0.972±0.004 for 5-fold-CV and 5 iter
dots Size: (1024, 2) Method: XGB Acc: 0.978±0.003 for 5-fold-CV and 5 iter

Sept. 5 scaler = 14
and Size: (500, 2) Method: svm_classifier Acc: 0.921±0.002 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: svm_classifier Acc: 0.919±0.004 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: svm_classifier Acc: 0.991±0.007 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: svm_classifier Acc: 0.632±0.019 for 5-fold-CV and 5 iter
checkerboard Size: (1024, 2) Method: svm_classifier Acc: 0.554±0.000 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: svm_classifier Acc: 0.690±0.000 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: svm_classifier Acc: 0.819±0.006 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: svm_classifier Acc: 0.694±0.000 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: svm_classifier Acc: 0.762±0.000 for 5-fold-CV and 5 iter
and Size: (500, 2) Method: bayes_classifier Acc: 0.958±0.004 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: bayes_classifier Acc: 0.940±0.003 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: bayes_classifier Acc: 0.967±0.009 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: bayes_classifier Acc: 0.566±0.014 for 5-fold-CV and 5 iter
checkerboard Size: (1024, 2) Method: bayes_classifier Acc: 0.508±0.007 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: bayes_classifier Acc: 0.830±0.006 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: bayes_classifier Acc: 0.819±0.007 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: bayes_classifier Acc: 0.691±0.005 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: bayes_classifier Acc: 0.762±0.000 for 5-fold-CV and 5 iter
and Size: (500, 2) Method: MLP Acc: 0.947±0.009 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: MLP Acc: 0.924±0.007 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: MLP Acc: 1.000±0.000 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: MLP Acc: 0.956±0.033 for 5-fold-CV and 5 iter
checkerboard Size: (1024, 2) Method: MLP Acc: 0.562±0.017 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: MLP Acc: 0.893±0.058 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: MLP Acc: 0.831±0.015 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: MLP Acc: 0.767±0.031 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: MLP Acc: 0.774±0.012 for 5-fold-CV and 5 iter
and Size: (500, 2) Method: XGB Acc: 1.000±0.000 for 5-fold-CV and 5 iter
or Size: (500, 2) Method: XGB Acc: 1.000±0.000 for 5-fold-CV and 5 iter
linear Size: (500, 2) Method: XGB Acc: 0.982±0.005 for 5-fold-CV and 5 iter
xor Size: (500, 2) Method: XGB Acc: 1.000±0.000 for 5-fold-CV and 5 iter
checkerboard Size: (1024, 2) Method: XGB Acc: 0.994±0.002 for 5-fold-CV and 5 iter
circle Size: (500, 2) Method: XGB Acc: 0.988±0.006 for 5-fold-CV and 5 iter
sine Size: (500, 2) Method: XGB Acc: 0.983±0.006 for 5-fold-CV and 5 iter
concentric Size: (500, 2) Method: XGB Acc: 0.988±0.004 for 5-fold-CV and 5 iter
dots Size: (500, 2) Method: XGB Acc: 0.971±0.006 for 5-fold-CV and 5 iter



# Reservoir Computing

## AMN-QP to create a reservoir for various species

In [ ]:
# Create, train and evaluate AMN_QP models
# on GR FBA simulated training set
# Repeat the process with different seeds
# This cell takes several hours to execute
# Save the best model in a reservoir

from Library.Import import *
from Library.Build_Model import Neural_Model, model_input
from Library.Build_Model import evaluate_model, train_evaluate_model
from sklearn.metrics import r2_score

DIRECTORY = './'
seed = 10
np.random.seed(seed=seed)
species = [ 
    'e_coli_core', #0
    'iEK1008', 
    'iIT341', #2
    'iJN1463', 
    'iML1515', #4
    'iMM904',
    'iPC815', #6
    'iYO844',
    'iYS1720', 
    'iYS854', # 9
    'iML1515EXP' #10
]

model_type = 'AMN_QP' # 'AMN_QP' 'ANN_Dense'

for i in [4]: # list(range(11))    
    trainname = f'{species[i]}_train'
    trainingfile = f'{DIRECTORY}Dataset_input/{trainname}'
    reservoirname = f'{trainname}_{model_type}'
    reservoirfile = f'{DIRECTORY}Reservoir/{reservoirname}'
    if 'EXP' in trainname:
        n_hidden, hidden_dim, epochs, niter = 1, 500, 1000, 0 # AMN 1, 500, 1000 / 500, 0 ANN Dense 0, 0, 100, 0
    else:
        n_hidden, hidden_dim, epochs, niter = 1, 100, 100, 1
    Maxloop, Q2, PRED = 3, [], [] # 5, [], []
    
    # Training
    for Nloop in range(Maxloop):
        model = Neural_Model(trainingfile=trainingfile,
                             model_type=model_type,
                             scaler=True,
                             n_hidden=n_hidden, hidden_dim=hidden_dim,
                             output_dim=1,
                             epochs=epochs, 
                             xfold=5, niter=niter,
                             verbose=False)
        model.batch_size = 100 if model.X.shape[0] > 1000 else 10
        print(f'{species[i]} Y {model.Y.shape} minY {np.min(model.Y):.2f} '
              f'maxY {np.max(model.Y):.2f} Objective {model.objective} '
              f'Unique {len(set(list(model.Y[:,0])))}')
        
        # Train and evaluate
        start_time = time.time()
        reservoir, pred, stats, _ = train_evaluate_model(model, verbose=True)
        delta_time = time.time() - start_time

        # Printing cross-validation results
        stats.printout(reservoirname, delta_time)
        r2 = r2_score(model.Y[:,0], pred[:,0], multioutput='variance_weighted')
        print(f'Iter {Nloop} Collated Q2 growth {r2:.4f}')
        r2 = r2_score(model.Y, pred[:,:model.Y.shape[1]], multioutput='variance_weighted')
        print(f'Iter {Nloop} Collated Q2 all {r2:.4f}')
        Q2.append(r2)
        PRED.append(pred[:, 0])
        if r2 == max(Q2):  # save the best model
            reservoir.save(reservoirfile)

    # Some printing
    Q2, PRED = np.asarray(Q2), np.asarray(PRED)
    print(f'{trainname} Averaged Q2 = {np.mean(Q2):4f} ± {np.std(Q2):.4f} Best Q2 = {np.max(Q2):.4f}')
    reservoir.load(reservoirfile, output_dim=1)
    reservoir.printout()
    X, Y = model_input(reservoir, verbose=False)
    print(X.shape, Y.shape)
    pred, _ = evaluate_model(reservoir.model, X, Y, reservoir, verbose=False)
    y = pred[:,:model.Y.shape[1]]     
    r2 = r2_score(model.Y[:,0], y[:,0], multioutput='variance_weighted')
    print(f'Final R2 growth {r2:.4f}')


iML1515 Y (500, 1) minY 0.00 maxY 0.00 Objective None Unique 1
AMN scaler: 6.5
QP input shape: (500, 28) (500, 4)
-------train (400, 28) (400, 4)
-------test  (100, 28) (100, 4)
----------------------------------- AMN_QP
Dense layer n_hidden: 1 hidden_dim: 100 input_dim: 28 output_dim: 3682 activation: relu trainable: True
Instructions for updating:
Use `tf.linalg.matmul` instead
AMN output shapes for PoutV: (None, 3682) SV: (None, 1) Pin: (None, 1) Pko: (None, 1)  V: (None, 3682) outputs: (None, 7367)
nbr parameters: 374782
13/13 [==============================] - 0s 7ms/step
Loss Vout: 3.8E-04
Loss SV:   1.0E-02
Loss Vin:  2.6E-03
Loss Vko:  0.0E+00
13/13 [==============================] - 0s 8ms/step
Loss Vout: 5.3E-08
Loss SV:   7.1E-07
Loss Vin:  7.3E-08
Loss Vko:  0.0E+00
4/4 [==============================] - 0s 10ms/step
Loss Vout: 5.4E-08
Loss SV:   7.6E-07
Loss Vin:  6.0E-08
Loss Vko:  0.0E+00
train = 1.00 test = 1.00 loss-train = 0.000000 loss-test = 0.000000 Media found = 0

## RC for Regression / Classification

### Regression

In [2]:
# Create, train and evaluate RC models
# on ML regression problems

from Library.Import import *
from Library.Utilities import read_XY
from Library.Build_Reservoir import RC_run, RC_write_multiple

DIRECTORY = './'
problems = [
    'raw_geographical_origin_of_music', # 0
    'raw_pumadyn32nh',
    'raw_white_wine', # 2 *
    'raw_kin8nm', # 3 *
    'raw_cars', # 4 *
    'raw_airfoil_self_noise', # 5 *
    'raw_QSAR_fish_toxicity', # 6 *
    'raw_space_ga', #7 *
    'raw_concrete_compressive_strength', # 8 *
    'raw_grid_stability', # 9 *
    'raw_miami_housing', # 10
    'raw_cpu_activity', # 11 *
    'raw_naval_propulsion_plant', # 12
    'raw_energy_efficiency' # 13 *
]
species = [ 
    'e_coli_core', #0
    'iEK1008', 
    'iIT341', #2
    'iJN1463', 
    'iML1515', #4
    'iMM904',
    'iPC815', #6
    'iYO844',
    'iYS1720', 
    'iYS854', #9
    'iML1515EXP', #10
]

seed = 1
np.random.seed(seed=seed)
xfold = 5
repeat = 3 # 3
precision = 0
train_rate = 1.0e-4
n_hidden_prior = -1 # -1 binary input,  >0 ANN 
hidden_dim_prior = 0
activation_prior='' # '' or 'sharp_sigmoid' or 'relu'
n_hidden_post = -1 #  -1 a scaler is applied, >0 a ANN is used
hidden_dim_post = 0
temperature = False
multiple = -1 # -1 no stats > 0 nbr of reservoirs to get stats 
weight_pred_true_media = 0 # Loss to collect already generated media
failure = 10

run = 'selective_obj' # (fixed, generative, selective) x (obj, phe)
    
if run == 'generative_phe': 
    mode = 'AMN_phenotype' 
    epochs = 500
    n_hidden_prior = 1 
    hidden_dim_prior = 28 
    activation_prior='sharp_sigmoid'
    n_hidden_post = 1 
    hidden_dim_post = 500

if run == 'fixed_phe':
    mode = 'AMN_phenotype' 
    epochs = 500
    n_hidden_post = 1 #  0
    hidden_dim_post = 500

if run == 'generative_obj': 
    mode = 'AMN_objective'
    epochs = 500
    n_hidden_prior = 1
    hidden_dim_prior = 28 
    activation_prior='sharp_sigmoid' 
    weight_pred_true_media = 0.5

if run == 'selective_obj': # For iML1515EXP only
    mode = 'AMN_objective'
    epochs = 1000
    n_hidden_prior = 3 
    hidden_dim_prior = 280
    activation_prior='gumbel_softmax' 
    weight_pred_true_media = 0
    n_hidden_post = -1 # 0 or -1
    temperature = True
    
for i in [10]: # range(11)
    s = species[i]
    reservoirname = f'{s}_train_AMN_QP' # for iML1515EXP : _train_AMN_QP_10
    reservoirfile = f'{DIRECTORY}Reservoir/{reservoirname}' 
    for j in [13]: # range(14)
        problem = problems[j].split('.')[0]
        trainingfile = f'{DIRECTORY}Dataset_input/Dataset_open_ML/Regression/{problem}'
        if 'fixed' in run:
            trainingfile = f'{trainingfile}_{s}_binary'
        resultfile = f'{DIRECTORY}Result/{problem}_{s}_{run}'
        H, X, Y = read_XY(trainingfile, nY=1, scaling='XY')
        start_time = time.time()
        model, pred, R2_avr, R2_dev, \
        Q2_avr, Q2_dev, Q2_best, Med = \
        RC_run(reservoirfile, X, Y,
               mode=mode, 
               n_hidden_prior=n_hidden_prior, 
               n_hidden_post=n_hidden_post,
               hidden_dim_prior=hidden_dim_prior,
               hidden_dim_post=hidden_dim_post, 
               activation_prior=activation_prior,
               train_rate=train_rate, 
               precision=precision,
               temperature=temperature,
               failure=failure,
               weight_pred_true_media=weight_pred_true_media,
               repeat=repeat, xfold=xfold, epochs=epochs, verbose=True)
        delta_time = time.time() - start_time
        print(f'{problem} {s} {run} hidden-size: {hidden_dim_prior} \
R2: {R2_avr:.4f} ± {R2_dev:.4f} Q2: {Q2_avr:.4f} ± {Q2_dev:.4f} Q2-best: {Q2_best:.4f} cpu time {delta_time:.2f} Media found {Med:.2f}')
        RC_write_multiple(reservoirfile, resultfile, model, 
                          Y, pred, 
                          multiple=multiple, 
                          precision_X=True, verbose=False)


number of reactions: 3682=1
number of metabolites: 1877
filtered measurements size: 1
RC reservoir file: ./Reservoir/iML1515EXP_train_AMN_QP
RC model type: RC
RC number constraint: 1
RC precsion: 0
RC model input dim: 8
RC model output dim: 1
training set size (768, 8) (768, 1)
reservoir S, Pin, Pout matrices (1877, 3682) (28, 3682) (1, 3682)
RC training epochs: 1000
RC training scoring: r2_score
RC training learn rate: 0.0001
RC training dropout: 0.25
RC training batch size: 10
RC training validation iter: 0
RC training xfold: 5
--------prior network --------
training file: None
model type: ANN_Dense
model number constraints: 0
model scaler: 0.0
model input dim: 8
model output dim: 280
no training set provided
nbr hidden layer: 3
hidden layer size: 280
activation function: gumbel_softmax
--------reservoir network-----
training file: ./Dataset_input/iML1515EXP_train
model type: AMN_QP
model number constraints: 3
model scaler: 1.0
model input dim: 28
model output dim: 1
training set siz

### Classification

train_rate = 1.0e-3 = NO
n_hidden_post =  -1 drop of 10 below from before NO
n_hidden_post = 0 drop to 0.79 NO
weight_pred_true_media = 0 = back to 0.92 YES

In [2]:
# Create, train and evaluate RC models
# on ML classification problems

from Library.Import import *
from Library.Utilities import read_XY
from Library.Build_Reservoir import RC_run, RC_write_multiple
from sklearn.metrics import r2_score, accuracy_score, f1_score, matthews_corrcoef

DIRECTORY = './'
problems = [
    'and', #0
    'or',  
    'linear', #2
    'triangle', #3
    'xor',    
    'checkerboard', #5
    'circle',
    'sine',  #7
    'concentric', #8
    'dots' #9
]

species = [ 
    'e_coli_core', #0
    'iEK1008', 
    'iIT341', #2
    'iJN1463', 
    'iML1515', #4
    'iMM904',
    'iPC815', #6
    'iYO844',
    'iYS1720', #8
    'iYS854', 
    'iML1515EXP' #10
]

seed = 1
np.random.seed(seed=seed)
xfold = 5 # 10 # 25 gives 5 in test set
repeat = 3 # 3
precision = 0
train_rate = 1.0e-4
temperature = False
multiple = 1 # -1 no stats > 0 nbr of reservoirs to get stats 
weight_pred_true_media = 0 # Loss to collect already generated media
failure = 10

print("TensorFlow version:", tf.__version__)

run = 'generative_obj' # (fixed, generative, selective) x (obj, phe)
scoring_function = accuracy_score

if run == 'generative_phe': 
    mode = 'AMN_phenotype' 
    epochs = 500
    n_hidden_prior = 1 
    hidden_dim_prior = 28 
    activation_prior='sharp_sigmoid'
    n_hidden_post = 1 
    hidden_dim_post = 500

if run == 'fixed_phe':
    mode = 'AMN_phenotype' 
    epochs = 500
    n_hidden_prior = -1
    hidden_dim_prior = -1 
    activation_prior=''
    n_hidden_post = 1 
    hidden_dim_post = 500

if run == 'generative_obj': 
    mode = 'AMN_objective'
    epochs = 500
    n_hidden_prior = 1
    hidden_dim_prior = 28 
    activation_prior='sharp_sigmoid' 
    n_hidden_post = -1 
    hidden_dim_post = 0
    weight_pred_true_media = 0.5

if run == 'selective_obj': # For iML1515EXP only
    mode = 'AMN_objective'
    epochs = 1000
    n_hidden_prior = 3 
    hidden_dim_prior = 280 # 280
    activation_prior='gumbel_softmax' 
    n_hidden_post = -1 
    hidden_dim_post = 0
    weight_pred_true_media = 1
    temperature = True

for i in [10]: # range(11)
        s = species[i]
        reservoirname = f'{s}_train_AMN_QP' 
        reservoirfile = f'{DIRECTORY}Reservoir/{reservoirname}' 
        for j in [8]: # range(10):
            problem = problems[j].split('.')[0]
            trainingfile = f'{DIRECTORY}Dataset_input/Dataset_classification/{problem}'
            if 'fixed' in run: 
                trainingfile = f'{trainingfile}_{s}_binary'
            resultfile = f'{DIRECTORY}Result/{problem}_{s}_{run}'
            H, X, Y = read_XY(trainingfile, nY=1, scaling='XY')
            start_time = time.time()
            model, pred, Acc_train_avr, Acc_train_dev, \
            Acc_test_avr, Acc_test_dev, Acc_best, Med = \
            RC_run(reservoirfile, X, Y,
                   mode=mode, 
                   scoring_function=scoring_function,
                   n_hidden_prior=n_hidden_prior, 
                   n_hidden_post=n_hidden_post,
                   hidden_dim_prior=hidden_dim_prior,
                   hidden_dim_post=hidden_dim_post, 
                   activation_prior=activation_prior,
                   train_rate=train_rate, 
                   precision=precision,
                   temperature=temperature,
                   failure=failure,
                   weight_pred_true_media=weight_pred_true_media,
                   repeat=repeat, xfold=xfold, epochs=epochs, verbose=False)
            delta_time = time.time() - start_time
            print(f'{problem} {s} {run} hidden-size: {hidden_dim_prior} \
Acc-train: {Acc_train_avr:.4f} ± {Acc_train_dev:.4f} Acc-test: {Acc_test_avr:.4f} ± {Acc_test_dev:.4f} \
Acc-best: {Acc_best:.4f} cpu time {delta_time:.2f} Media found {Med:.2f}')
            RC_write_multiple(reservoirfile, resultfile, model, 
                              Y, pred, 
                              multiple=multiple, 
                              precision_X=True, verbose=False)


TensorFlow version: 2.16.2


ValueError: Unrecognized keyword arguments passed to Lambda: {'function_type': 'function'}

triangle iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9870 ± 0.0059 Acc-test: 0.9860 ± 0.0049 Acc-best: 0.9920 cpu time 5315.92 Media found 1.00

triangle iML1515EXP generative_obj hidden-size: 28 Acc-train: 0.9300 ± 0.0080 Acc-test: 0.9147 ± 0.0098 Acc-best: 0.9260 cpu time 5586.15 Media found 0.24


and iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.8665 ± 0.0456 Acc-test: 0.8573 ± 0.0520 Acc-best: 0.8980 cpu time 10955.78 Media found 1.00 (seed=1)
and iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.8807 ± 0.0603 Acc-test: 0.8847 ± 0.0682 Acc-best: 0.9540 cpu time 7663.29 Media found 1.00 (seed=10)
and iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9428 ± 0.0091 Acc-test: 0.9300 ± 0.0216 Acc-best: 0.9500 cpu time 4222.45 Media found 1.00 (seed=2)

or iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9777 ± 0.0033 Acc-test: 0.9733 ± 0.0025 Acc-best: 0.9760 cpu time 7979.80 Media found 1.00
linear iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9968 ± 0.0013 Acc-test: 0.9927 ± 0.0025 Acc-best: 0.9960 cpu time 4366.30 Media found 1.00

xor iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.8668 ± 0.0081 Acc-test: 0.8580 ± 0.0098 Acc-best: 0.8700 cpu time 5152.33 Media found 1.00
checkerboard iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.5642 ± 0.0087 Acc-test: 0.5653 ± 0.0104 Acc-best: 0.5800 cpu time 4517.76 Media found 1.00
circle iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.8522 ± 0.0508 Acc-test: 0.8507 ± 0.0465 Acc-best: 0.9160 cpu time 4423.94 Media found 1.00
sine iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9328 ± 0.0012 Acc-test: 0.9220 ± 0.0086 Acc-best: 0.9300 cpu time 4597.10 Media found 1.00


concentric iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.6880 ± 0.0018 Acc-test: 0.6833 ± 0.0025 Acc-best: 0.6860 cpu time 4527.74 Media found 1.00 (seed=1)
concentric iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.6807 ± 0.0038 Acc-test: 0.6820 ± 0.0057 Acc-best: 0.6900 cpu time 4414.25 Media found 1.00 (seed=2)
concentric iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.6840 ± 0.0042 Acc-test: 0.6893 ± 0.0109 Acc-best: 0.7040 cpu time 4844.88 Media found 1.00  1000

dots iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.7842 ± 0.0133 Acc-test: 0.7700 ± 0.0085 Acc-best: 0.7760 cpu time 5181.59 Media found 1.00 (seed=1)
dots iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.7883 ± 0.0045 Acc-test: 0.7593 ± 0.0222 Acc-best: 0.7880 cpu time 9566.44 Media found 1.00 (seed=2)

Sept. 6 2024 - 500 datapoints scaler = 13


and iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.9663 ± 0.0029 Acc-test: 0.9460 ± 0.0033 Acc-best: 0.9500 cpu time 1579.20 Media found 0.02
or iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.9788 ± 0.0016 Acc-test: 0.9547 ± 0.0093 Acc-best: 0.9640 cpu time 1903.24 Media found 0.00
linear iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.9302 ± 0.0072 Acc-test: 0.8807 ± 0.0084 Acc-best: 0.8920 cpu time 2125.35 Media found 0.01
xor iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.8432 ± 0.0117 Acc-test: 0.7613 ± 0.0120 Acc-best: 0.7780 cpu time 1707.51 Media found 0.00
checkerboard iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.8817 ± 0.0079 Acc-test: 0.8260 ± 0.0102 Acc-best: 0.8360 cpu time 1930.78 Media found 0.01
circle iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.9288 ± 0.0072 Acc-test: 0.8720 ± 0.0247 Acc-best: 0.9040 cpu time 2195.07 Media found 0.01
sine iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.9170 ± 0.0032 Acc-test: 0.8707 ± 0.0034 Acc-best: 0.8740 cpu time 2000.94 Media found 0.01
concentric iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.9017 ± 0.0031 Acc-test: 0.8527 ± 0.0175 Acc-best: 0.8760 cpu time 2050.19 Media found 0.01
dots iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.8883 ± 0.0121 Acc-test: 0.8307 ± 0.0161 Acc-best: 0.8440 cpu time 2122.95 Media found 0.00

and iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9343 ± 0.0454 Acc-test: 0.9180 ± 0.0525 Acc-best: 0.9920 cpu time 8457.44 Media found 1.00
or iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9755 ± 0.0045 Acc-test: 0.9727 ± 0.0098 Acc-best: 0.9840 cpu time 4315.42 Media found 1.00
linear iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9965 ± 0.0015 Acc-test: 0.9953 ± 0.0009 Acc-best: 0.9960 cpu time 4586.91 Media found 1.00
xor iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.8165 ± 0.0753 Acc-test: 0.8227 ± 0.0764 Acc-best: 0.9100 cpu time 4780.87 Media found 1.00
checkerboard iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.5388 ± 0.0157 Acc-test: 0.5187 ± 0.0319 Acc-best: 0.5560 cpu time 5064.82 Media found 1.00
circle iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.8038 ± 0.0461 Acc-test: 0.7960 ± 0.0540 Acc-best: 0.8720 cpu time 4862.88 Media found 1.00
sine iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9243 ± 0.0031 Acc-test: 0.9153 ± 0.0052 Acc-best: 0.9200 cpu time 4716.44 Media found 1.00
concentric iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.6882 ± 0.0144 Acc-test: 0.6840 ± 0.0085 Acc-best: 0.6960 cpu time 4445.04 Media found 1.00
dots iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.7867 ± 0.0033 Acc-test: 0.7567 ± 0.0019 Acc-best: 0.7580 cpu time 4312.38 Media found 1.00

and iML1515EXP generative_obj hidden-size: 28 Acc-train: 0.9217 ± 0.0080 Acc-test: 0.9127 ± 0.0090 Acc-best: 0.9200 cpu time 2089.84 Media found 0.16
or iML1515EXP generative_obj hidden-size: 28 Acc-train: 0.9452 ± 0.0159 Acc-test: 0.9433 ± 0.0180 Acc-best: 0.9580 cpu time 2093.52 Media found 0.29
linear iML1515EXP generative_obj hidden-size: 28 Acc-train: 0.9965 ± 0.0007 Acc-test: 0.9920 ± 0.0071 Acc-best: 0.9980 cpu time 2138.60 Media found 0.21
xor iML1515EXP generative_obj hidden-size: 28 Acc-train: 0.8187 ± 0.0193 Acc-test: 0.8247 ± 0.0181 Acc-best: 0.8480 cpu time 2240.34 Media found 0.29
checkerboard iML1515EXP generative_obj hidden-size: 28 Acc-train: 0.6045 ± 0.0330 Acc-test: 0.5793 ± 0.0189 Acc-best: 0.6040 cpu time 2140.38 Media found 0.18
circle iML1515EXP generative_obj hidden-size: 28 Acc-train: 0.7610 ± 0.0231 Acc-test: 0.7520 ± 0.0255 Acc-best: 0.7880 cpu time 2099.28 Media found 0.35
sine iML1515EXP generative_obj hidden-size: 28 Acc-train: 0.8968 ± 0.0033 Acc-test: 0.8980 ± 0.0085 Acc-best: 0.9040 cpu time 2083.09 Media found 0.37
concentric iML1515EXP generative_obj hidden-size: 28 Acc-train: 0.6780 ± 0.0000 Acc-test: 0.6780 ± 0.0000 Acc-best: 0.6780 cpu time 1939.71 Media found 0.12
dots iML1515EXP generative_obj hidden-size: 28 Acc-train: 0.7420 ± 0.0000 Acc-test: 0.7420 ± 0.0000 Acc-best: 0.7420 cpu time 1809.44 Media found 0.04



Sept. 6 1000 datapoints scaler = 13

and iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.9953 ± 0.0013 Acc-test: 0.9907 ± 0.0024 Acc-best: 0.9940 cpu time 3772.09 Media found 0.01
or iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.9953 ± 0.0022 Acc-test: 0.9853 ± 0.0012 Acc-best: 0.9870 cpu time 4053.57 Media found 0.01
linear iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.9564 ± 0.0011 Acc-test: 0.9357 ± 0.0025 Acc-best: 0.9390 cpu time 3943.10 Media found 0.00
xor iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.9345 ± 0.0023 Acc-test: 0.9013 ± 0.0026 Acc-best: 0.9050 cpu time 4038.54 Media found 0.00
checkerboard iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.9089 ± 0.0071 Acc-test: 0.8823 ± 0.0127 Acc-best: 0.8950 cpu time 3942.11 Media found 0.01
circle iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.9577 ± 0.0020 Acc-test: 0.9347 ± 0.0017 Acc-best: 0.9370 cpu time 4008.72 Media found 0.01
sine iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.9531 ± 0.0041 Acc-test: 0.9343 ± 0.0077 Acc-best: 0.9450 cpu time 3953.03 Media found 0.00
concentric iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.9468 ± 0.0007 Acc-test: 0.9200 ± 0.0036 Acc-best: 0.9250 cpu time 4273.63 Media found 0.02
dots iML1515EXP fixed_phe hidden-size: 0 Acc-train: 0.8834 ± 0.0084 Acc-test: 0.8523 ± 0.0084 Acc-best: 0.8610 cpu time 4186.83 Media found 0.01

and iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9463 ± 0.0410 Acc-test: 0.9427 ± 0.0449 Acc-best: 0.9980 cpu time 17398.42 Media found 1.00
or iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9918 ± 0.0001 Acc-test: 0.9883 ± 0.0009 Acc-best: 0.9890 cpu time 11185.17 Media found 1.00
linear iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9971 ± 0.0001 Acc-test: 0.9957 ± 0.0017 Acc-best: 0.9980 cpu time 10498.69 Media found 1.00
xor iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.8874 ± 0.0591 Acc-test: 0.8770 ± 0.0542 Acc-best: 0.9240 cpu time 10646.57 Media found 1.00
checkerboard iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.6046 ± 0.0298 Acc-test: 0.5917 ± 0.0134 Acc-best: 0.6040 cpu time 9339.83 Media found 1.00

Sept. 5 scaler = 32 1000 datapoints

and iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9449 ± 0.0355 Acc-test: 0.9463 ± 0.0332 Acc-best: 0.9854 cpu time 2003.27 Media found 1.00
or iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9673 ± 0.0010 Acc-test: 0.9668 ± 0.0008 Acc-best: 0.9678 cpu time 2011.87 Media found 1.00
linear iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.9991 ± 0.0005 Acc-test: 0.9980 ± 0.0021 Acc-best: 1.0000 cpu time 1912.48 Media found 1.00
xor iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.8393 ± 0.0423 Acc-test: 0.8451 ± 0.0399 Acc-best: 0.8926 cpu time 1790.61 Media found 1.00
checkerboard iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.5511 ± 0.0037 Acc-test: 0.5456 ± 0.0115 Acc-best: 0.5537 cpu time 1699.78 Media found 1.00
circle iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.8198 ± 0.0763 Acc-test: 0.8154 ± 0.0742 Acc-best: 0.9053 cpu time 1586.40 Media found 1.00
sine iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.8951 ± 0.0121 Acc-test: 0.8864 ± 0.0134 Acc-best: 0.9053 cpu time 1604.47 Media found 1.00
concentric iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.6943 ± 0.0000 Acc-test: 0.6943 ± 0.0000 Acc-best: 0.6943 cpu time 1604.50 Media found 1.00
dots iML1515EXP selective_obj hidden-size: 280 Acc-train: 0.7806 ± 0.0068 Acc-test: 0.7741 ± 0.0095 Acc-best: 0.7871 cpu time 1615.61 Media found 1.00

and iML1515EXP fixed_phe hidden-size: 280 Acc-train: 0.7584 ± 0.0008 Acc-test: 0.7572 ± 0.0009 Acc-best: 0.7578 cpu time 1228.13 Media found 1.00
or iML1515EXP fixed_phe hidden-size: 280 Acc-train: 0.7412 ± 0.0000 Acc-test: 0.7412 ± 0.0000 Acc-best: 0.7412 cpu time 1201.38 Media found 0.66
linear iML1515EXP fixed_phe hidden-size: 280 Acc-train: 0.5081 ± 0.0028 Acc-test: 0.5111 ± 0.0051 Acc-best: 0.5156 cpu time 1233.97 Media found 1.00
xor iML1515EXP fixed_phe hidden-size: 280 Acc-train: 0.5043 ± 0.0025 Acc-test: 0.4870 ± 0.0091 Acc-best: 0.4961 cpu time 1202.98 Media found 0.61
checkerboard iML1515EXP fixed_phe hidden-size: 280 Acc-train: 0.5537 ± 0.0000 Acc-test: 0.5537 ± 0.0000 Acc-best: 0.5537 cpu time 1236.81 Media found 0.43
circle iML1515EXP fixed_phe hidden-size: 280 Acc-train: 0.7236 ± 0.0000 Acc-test: 0.7236 ± 0.0000 Acc-best: 0.7236 cpu time 1164.19 Media found 0.53
sine iML1515EXP fixed_phe hidden-size: 280 Acc-train: 0.5041 ± 0.0046 Acc-test: 0.4883 ± 0.0156 Acc-best: 0.5068 cpu time 1186.05 Media found 0.98
concentric iML1515EXP fixed_phe hidden-size: 280 Acc-train: 0.6943 ± 0.0000 Acc-test: 0.6943 ± 0.0000 Acc-best: 0.6943 cpu time 1198.61 Media found 0.64
dots iML1515EXP fixed_phe hidden-size: 280 Acc-train: 0.7617 ± 0.0000 Acc-test: 0.7617 ± 0.0000 Acc-best: 0.7617 cpu time 1222.93 Media found 1.00

NEW (Sept)  fixed: hidden-layer=(1,500) - all: xfold=5

and iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.9116 ± 0.0179 Acc-best: 0.9242 cpu time 511.70 Media found 0.02
checkerboard iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.7222 ± 0.0036 Acc-best: 0.7273 cpu time 524.42 Media found 0.00
circle iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.8157 ± 0.0129 Acc-best: 0.8333 cpu time 527.08 Media found 0.02
concentric iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.7525 ± 0.0156 Acc-best: 0.7727 cpu time 531.37 Media found 0.01
dots iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.7677 ± 0.0305 Acc-best: 0.8106 cpu time 535.86 Media found 0.02
linear iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.8258 ± 0.0214 Acc-best: 0.8561 cpu time 538.72 Media found 0.00
or iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.8030 ± 0.0164 Acc-best: 0.8182 cpu time 568.20 Media found 0.02
sine iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.8030 ± 0.0186 Acc-best: 0.8258 cpu time 578.39 Media found 0.01
xor iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.6061 ± 0.0309 Acc-best: 0.6439 cpu time 605.36 Media found 0.03

NEW (Sept)  fixed: hidden-layer=(1,500) - all: xfold=10

and iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.9061 ± 0.0077 Acc-best: 0.9167 cpu time 1681.45 Media found 0.02
checkerboard iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.7485 ± 0.0088 Acc-best: 0.7652 cpu time 1942.22 Media found 0.00
circle iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.8424 ± 0.0175 Acc-best: 0.8712 cpu time 2207.11 Media found 0.02
concentric iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.7455 ± 0.0091 Acc-best: 0.7576 cpu time 2201.53 Media found 0.01
dots iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.8061 ± 0.0077 Acc-best: 0.8182 cpu time 1877.40 Media found 0.02
linear iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.8333 ± 0.0083 Acc-best: 0.8485 cpu time 1991.41 Media found 0.00
or iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.8197 ± 0.0211 Acc-best: 0.8561 cpu time 2167.28 Media found 0.02
sine iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.8136 ± 0.0183 Acc-best: 0.8485 cpu time 1928.02 Media found 0.01
xor iML1515EXP fixed_phe hidden-size: 0 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.6394 ± 0.0170 Acc-best: 0.6591 cpu time 2112.88 Media found 0.03

OLD (Aug) fixed: hidden-layer=(0,500) - all: xfold=5

and iML1515EXP selective-obj hidden-size: 280 R2: 1.00 ± 0.0000 Q2: 0.89 ± 0.0270 cpu time 1791.76 Media found 1.00
checkerboard iML1515EXP selective-obj hidden-size: 280 R2: 1.00 ± 0.0000 Q2: 0.51 ± 0.0062 cpu time 1706.20 Media found 1.00
circle iML1515EXP selective-obj hidden-size: 280 R2: 1.00 ± 0.0000 Q2: 0.73 ± 0.0214 cpu time 1660.93 Media found 1.00
concentric iML1515EXP selective-obj hidden-size: 280 R2: 1.00 ± 0.0000 Q2: 0.73 ± 0.0000 cpu time 1530.27 Media found 1.00
dots iML1515EXP selective-obj hidden-size: 280 R2: 1.00 ± 0.0000 Q2: 0.76 ± 0.0062 cpu time 1498.49 Media found 1.00
linear iML1515EXP selective-obj hidden-size: 280 R2: 1.00 ± 0.0000 Q2: 0.99 ± 0.0036 cpu time 1428.93 Media found 1.00
or iML1515EXP selective-obj hidden-size: 280 R2: 1.00 ± 0.0000 Q2: 0.90 ± 0.0071 cpu time 1449.66 Media found 1.00
sine iML1515EXP selective-obj hidden-size: 280 R2: 1.00 ± 0.0000 Q2: 0.74 ± 0.0341 cpu time 1403.24 Media found 1.00
or iML1515EXP selective-obj hidden-size: 280 R2: 1.00 ± 0.0000 Q2: 0.65 ± 0.0129 cpu time 1402.04 Media found 1.00

and iML1515EXP selective_obj hidden-size: 280 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.7803 ± 0.0000 Acc-best: 0.7803 cpu time 1303.22 Media found 1.00
checkerboard iML1515EXP selective_obj hidden-size: 280 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.5455 ± 0.0214 Acc-best: 0.5606 cpu time 1570.03 Media found 1.00
circle iML1515EXP selective_obj hidden-size: 280 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.7576 ± 0.0000 Acc-best: 0.7576 cpu time 1619.83 Media found 1.00
concentric iML1515EXP selective_obj hidden-size: 280 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.7197 ± 0.0000 Acc-best: 0.7197 cpu time 1659.45 Media found 1.00
dots iML1515EXP selective_obj hidden-size: 280 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.7500 ± 0.0000 Acc-best: 0.7500 cpu time 1671.11 Media found 1.00
linear iML1515EXP selective_obj hidden-size: 280 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.9874 ± 0.0036 Acc-best: 0.9924 cpu time 1820.73 Media found 1.00
or iML1515EXP selective_obj hidden-size: 280 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.7955 ± 0.0000 Acc-best: 0.7955 cpu time 1725.67 Media found 1.00
sine iML1515EXP selective_obj hidden-size: 280 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.4520 ± 0.0143 Acc-best: 0.4621 cpu time 1799.13 Media found 1.00
xor iML1515EXP selective_obj hidden-size: 280 Acc-train: 1.0000 ± 0.0000 Acc-test: 0.5758 ± 0.0000 Acc-best: 0.5758 cpu time 1746.75 Media found 1.00

and iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.78 ± 0.0000 cpu time 413.12 Media found 0.00
checkerboard iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.56 ± 0.0094 cpu time 416.10 Media found 0.00
circle iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.76 ± 0.0000 cpu time 1740.21 Media found 0.00
concentric iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.71 ± 0.0036 cpu time 1725.73 Media found 0.00
dots iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.75 ± 0.0000 cpu time 417.31 Media found 0.00
linear iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.57 ± 0.0234 cpu time 420.46 Media found 0.00
or iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.80 ± 0.0000 cpu time 412.74 Media found 0.00
xor iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.58 ± 0.0036 cpu time 419.81 Media found 0.00

and iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.78 ± 0.0000 cpu time 413.27 Media found 0.00
checkerboard iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.56 ± 0.0036 cpu time 416.67 Media found 0.00
circle iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.76 ± 0.0000 cpu time 415.26 Media found 0.00
concentric iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.71 ± 0.0036 cpu time 416.54 Media found 0.00
dots iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.75 ± 0.0000 cpu time 424.04 Media found 0.00
linear iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.61 ± 0.0071 cpu time 421.57 Media found 0.00
or iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.80 ± 0.0000 cpu time 421.50 Media found 0.00
sine iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.56 ± 0.0327 cpu time 425.92 Media found 0.00
xor iML1515EXP fixed_phe hidden-size: 0 R2: 1.00 ± 0.0000 Q2: 0.58 ± 0.0000 cpu time 416.08 Media found 0.00
